<a href="https://colab.research.google.com/github/jiahaoChen/DemoTest/blob/master/%E5%9B%A0%E5%AD%90%E5%88%86%E6%9E%90_%E5%85%AC%E9%96%8B%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finlab > log.txt
!pip install ta-lib-everywhere > log.txt

In [ ]:
from finlab import data, backtest

marketcap = data.get('etl:market_value')
revenue   = data.get('monthly_revenue:當月營收')
close     = data.get('price:收盤價')

cond_smallcap = marketcap.rank(pct=True, axis=1) < 0.3
cond_revgro   = (revenue.average(3) / revenue.average(12)).rank(pct=True, axis=1) > 0.7
cond_momentum = (close / close.shift(20)).rank(pct=True, axis=1) > 0.7

pos = cond_smallcap & cond_revgro & cond_momentum
report = backtest.sim(pos, resample='ME', upload=False)
report.display()

輸入成功!
Daily usage: 28.1 / 5000 MB - etl:market_value
Daily usage: 31.8 / 5000 MB - monthly_revenue:當月營收
Daily usage: 58.0 / 5000 MB - price:收盤價
Daily usage: 93.7 / 5000 MB - etl:adj_close
Daily usage: 93.8 / 5000 MB - benchmark_return:發行量加權股價報酬指數
Daily usage: 93.9 / 5000 MB - security_categories
Daily usage: 130.8 / 5000 MB - price:成交股數
Daily usage: 132.0 / 5000 MB - etl:is_flagged_stock


<IPython.core.display.Javascript object>

In [ ]:
from finlab import data
import finlab.ml.feature as feature
import finlab.ml.label as label

features = feature.combine({
    'marketcap' : cond_smallcap,
    'revenue'   : cond_revgro,
    'momentum'  : cond_momentum
}, resample='ME')

labels = label.excess_over_mean(index=features.index, resample='ME')

In [ ]:
from finlab.tools.factor_analysis import calc_factor_return
from finlab.plot import plot_line

factor_return = calc_factor_return(features, labels).cumsum()
plot_line(factor_return, unit='.0%', title='因子累積報酬')

In [ ]:
from finlab.tools.factor_analysis import calc_centrality
centrality = calc_centrality(calc_factor_return(features, labels), 12)

plot_line(centrality, title='因子集中度')

Rolling Centrality: 149it [00:00, 406.20it/s]                         


In [ ]:
from finlab.tools.factor_analysis import calc_shapley_values
shapley = calc_shapley_values(features, labels)
plot_line(centrality, title='因子貢獻度')

Calculating Shapley values: 100%|██████████| 7/7 [00:03<00:00,  2.18it/s]


In [ ]:
from finlab.tools.factor_analysis import calc_ic

features_ic = {
    'marketcap': -marketcap,  # 小市值用負號
    'revenue'  : revenue.average(3) / revenue.average(12),
    'momentum' : close / close.shift(20),
}
features_ic = feature.combine(features_ic, resample='ME')

ic_df = calc_ic(features_ic, labels, rank=True)
plot_line(ic_df, title='因子相關性')

In [ ]:
from finlab.tools.factor_analysis import calc_regression_stats
centrality_trend = calc_regression_stats(centrality)
centrality_trend